In [1]:
import numpy as np

In [2]:
class Neuron:
    def __init__(self,value,_children=(),_op=''):
        # REMINDER HAPUS CAST KE INT, CAST HANYA UNTUK TESTING
        self.value = int(value);
        self._prev = set(_children);
        self._op = _op;
        self.grad = 0.0;

    def __repr__(self):
        stringVal = f"{self.value}";
        return stringVal

    def __add__(self,other):
        return Neuron(self.value + other.value,(self,other),'+');

    def __sub__(self,other):
        return Neuron(self.value - other.value,(self,other),'-');

    def __mul__(self,other):
        return Neuron(self.value * other.value,(self,other),'*');

    def __div__(self,other):
        return Neuron(self.value / other.value,(self,other),'/');

    def hello():
        print("hello")

    #TODO: Fungsi dan autograd



In [ ]:
class FFNN19:
    def __init__(self,
                 hidden_layer_count=1,
                 input_neuron_count=1,
                 output_neuron_count=1,
                 ):
        
        self.hidden_layer_count             = hidden_layer_count;
        self.input_neuron_count             = input_neuron_count;
        self.output_neuron_count            = output_neuron_count;
        self._weight_layer_count            = hidden_layer_count+1;
        self.weight_layers                  = [Neuron(0) for _ in range(self._weight_layer_count)];
        self.hidden_layers                  = [Neuron(0) for _ in range(hidden_layer_count)];
        self._activated_hidden_layers       = [Neuron(0) for _ in range(hidden_layer_count)];
        self._output_layer                  = [Neuron(0) for _ in range(output_neuron_count)];
        self._neuron_layer_set              = False;
        self._weight_layers_initialized     = False;
        self.outputs                        = []

    # Vectorize is not the most efficient solution (gw mager)
    def neuronGenerator(self,generator, *args, **kwargs):
        value = generator(*args, **kwargs)
        return np.vectorize(Neuron)(value)
    
    def setHiddenLayerNeuronCount(self,*args):
        if len(args)==0:
            for i in range(self.hidden_layer_count):
                self.hidden_layers[i] = self.neuronGenerator(np.zeros,(input())+1)
                self.hidden_layers[i][0] = Neuron(1)
            self._neuron_layer_set    = True
        elif len(args)==1:
            if len(args[0])!=self.hidden_layer_count:
                raise ValueError(f"Layer count mismatch: expected {self.hidden_layer_count} hidden layer, but {len(args[0])} were given")
            else:
                for i in range(self.hidden_layer_count):
                    self.hidden_layers[i] = self.neuronGenerator(np.zeros,args[0][i]+1)
                    self.hidden_layers[i][0] = Neuron(1)
                self._neuron_layer_set = True
        else:
            raise TypeError(f"FFNN19.setHiddenLayerNeuronCount() takes exactly 1 positional arguments but {len(args)} were given")

    def zeroWeightInitialization(self):
        if self._neuron_layer_set:
            for i in range(self._weight_layer_count):
                if i==0:
                    self.weight_layers[i] = self.neuronGenerator(np.zeros,(self.input_neuron_count+1,len(self.hidden_layers[i])))
                elif i==self._weight_layer_count-1:
                    self.weight_layers[i] = self.neuronGenerator(np.zeros,(len(self.hidden_layers[i-1]),self.output_neuron_count))
                else:
                    self.weight_layers[i] = self.neuronGenerator(np.zeros,(len(self.hidden_layers[i-1]),len(self.hidden_layers[i])))
            self._weight_layers_initialized     = True;
        else:
            raise ValueError(self._neuron_layer_set_message)

    def uniformWeightDistribution(self,lower,upper,seed):
        if self._neuron_layer_set:
            rng = np.random.default_rng(seed)
            for i in range(self._weight_layer_count):
                if i==0:
                    self.weight_layers[i] = self.neuronGenerator(rng.uniform,lower,upper,size=(self.input_neuron_count+1,len(self.hidden_layers[i])))
                elif i==self._weight_layer_count-1:
                    self.weight_layers[i] = self.neuronGenerator(rng.uniform,lower,upper,size=(len(self.hidden_layers[i-1]),self.output_neuron_count))
                else:
                    self.weight_layers[i] = self.neuronGenerator(rng.uniform,lower,upper,size=(len(self.hidden_layers[i-1]),len(self.hidden_layers[i])))
                if i!=self._weight_layer_count-1:
                    self.weight_layers[i][:,0] = Neuron(0)
            self._weight_layers_initialized     = True;
        else:
            raise ValueError(f"Neuron layer count not set, please set neuron layer count before weight initialization")
        
    def normalWeightDistribution(self,mean,variance,seed):
        if self._neuron_layer_set:
            rng = np.random.default_rng(seed)
            for i in range(self._weight_layer_count):
                if i==0:
                    self.weight_layers[i] = self.neuronGenerator(rng.normal,mean,variance,size=(self.input_neuron_count+1,len(self.hidden_layers[i])))
                elif i==self._weight_layer_count-1:
                    self.weight_layers[i] = self.neuronGenerator(rng.normal,mean,variance,size=(len(self.hidden_layers[i-1]),self.output_neuron_count))
                else:
                    self.weight_layers[i] = self.neuronGenerator(rng.normal,mean,variance,size=(len(self.hidden_layers[i-1]),len(self.hidden_layers[i])))
                if i!=self._weight_layer_count-1:
                    self.weight_layers[i][:,0] = Neuron(0)
            self._weight_layers_initialized     = True;
        else:
            raise ValueError(f"Neuron layer count not set, please set neuron layer count before weight initialization")

    # Single pass, masih bingung gmn caranya buat multiple
    def _feedforward(self,input_data):
        if self._weight_layers_initialized:
            input_data.insert(0,1)
            input_layer_neuron = self.neuronGenerator(np.array,object=input_data)
            for i in range(self._weight_layer_count):
                if i==0:
                    self.hidden_layers[i] = np.sum(input_layer_neuron*self.weight_layers[i].T,axis=1)
                    self.hidden_layers[i][0] = Neuron(1) 
                elif i==self._weight_layer_count-1:
                    self._output_layer = np.sum(self.hidden_layers[i-1]*self.weight_layers[i].T,axis=1)
                else:
                    self.hidden_layers[i] = np.sum(self.hidden_layers[i-1]*self.weight_layers[i].T,axis=1)
                    self.hidden_layers[i][0] = Neuron(1)
            self.outputs.append(self._output_layer)
        else:
            raise ValueError(f"Weight layers not initialized, please initialize weight layers before feedforward")

    # TODO, implementasi yg ini
    def _backpropagation(self):
        raise NotImplementedError("Belum diimplement")

    def fit(self,X,Y):
        raise NotImplementedError("Belum diimplement")
    
    def predict(self,X):
        raise NotImplementedError("Belum diimplement")
    

In [24]:
modelInstance = FFNN19(
    input_neuron_count=2,
    hidden_layer_count=3,
    output_neuron_count=2
    )

In [ ]:
modelInstance.setHiddenLayerNeuronCount([4,5,8])

In [28]:
modelInstance.uniformWeightDistribution(1,4,9)

In [31]:
modelInstance._feedforward([5,8])
print(1,5,7,"\n")

print(modelInstance.weight_layers[0],"\n")
print(modelInstance.hidden_layers[0],"\n")
print(modelInstance.weight_layers[1],"\n")
print(modelInstance.hidden_layers[1],"\n")
print(modelInstance.weight_layers[2],"\n")
print(modelInstance.hidden_layers[2],"\n")
print(modelInstance.weight_layers[3],"\n")
print(modelInstance._output_layer)

# print(modelInstance.weight_layers)

1 5 7 

[[0 1 2 3 3]
 [0 3 3 1 2]
 [0 1 1 3 3]] 

[1 24 25 32 37] 

[[0 1 3 1 3 1]
 [0 3 3 3 3 3]
 [0 3 1 2 2 2]
 [0 1 1 1 3 1]
 [0 2 2 3 1 3]] 

[1 254 206 266 258 266] 

[[0 1 1 2 2 1 1 3 3]
 [0 1 3 1 1 1 3 2 2]
 [0 2 1 3 1 2 1 1 3]
 [0 3 2 2 3 2 2 2 2]
 [0 3 2 2 2 3 3 1 3]
 [0 1 3 3 2 1 3 3 1]] 

[1 2505 2815 2720 2308 2239 3073 2305 2701] 

[[1 3]
 [2 1]
 [1 1]
 [2 2]
 [2 2]
 [2 2]
 [2 2]
 [3 3]
 [3 2]] 

[43524 38320]


In [34]:
print(modelInstance.outputs)
# print(modelInstance.outputs[1][0]._prev)


[array([43524, 38320], dtype=object), array([43524, 38320], dtype=object)]


In [6]:


# n_input_neuron = 3 # jumlah neuron input 
# n_input_instance = 1 # jumlah instance input
# n_hidden_layer = 2  # jumlah layer
# n_output_neuron = 1
# weight_layer_count = n_hidden_layer + 1


# input_layer_neuron = valueGenerator(np.random.randint,1,5,size=(n_input_instance,n_input_neuron))
# output_layer_neuron = valueGenerator(np.zeros,n_output_neuron)
# hidden_layer_neuron = [Value(0) for i in range(n_hidden_layer)]
# self.weight_layers = [Value(0) for i in range(weight_layer_count)]

# for i in range(n_hidden_layer):
#     hidden_layer_neuron[i] = np.zeros(int(input()))

# for i in range(weight_layer_count):
#     if i==0:
#         # weight_layer_neuron[i] = np.random.randint(5,size=(n_input_neuron,len(hidden_layer_neuron[i])))
#         self.weight_layers[i] = valueGenerator(np.random.randint,1,6,size=(n_input_neuron,len(hidden_layer_neuron[i])))
#     elif i==weight_layer_count-1:
#         self.weight_layers[i] = valueGenerator(np.random.randint,1,6,size=(len(hidden_layer_neuron[i-1]),n_output_neuron))
#         # weight_layer_neuron[i] = np.random.randint(5,size=(len(hidden_layer_neuron[i-1]),n_output_neuron))
#     else:
#         self.weight_layers[i] = valueGenerator(np.random.randint,1,6,size=(len(hidden_layer_neuron[i-1]),len(hidden_layer_neuron[i])))
#         # weight_layer_neuron[i] = np.random.randint(5,size=(len(hidden_layer_neuron[i-1]),len(hidden_layer_neuron[i])))

# for i in range(weight_layer_count):
#     if i==0:
#         hidden_layer_neuron[i] = np.sum(input_layer_neuron[0]*self.weight_layers[i].T,axis=1) 
#     elif i==weight_layer_count-1:
#         output_layer_neuron = np.sum(hidden_layer_neuron[i-1]*self.weight_layers[i].T,axis=1)
#     else:
#         hidden_layer_neuron[i] = np.sum(hidden_layer_neuron[i-1]*self.weight_layers[i].T,axis=1)

# print("Input")
# print(input_layer_neuron)
# print("Hidden Layer")
# print(hidden_layer_neuron)
# print("Output")
# print(output_layer_neuron)
# print("Weight")
# print(self.weight_layers)
# print("\n")

# print(type(self.weight_layers[0][0][0]))